<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/PersianRecognitionBot/persianRecognitionTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [2]:
dataset_path="/content/drive/MyDrive/Persian_Recognition"
width=height=224
batch_size=32
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.1
)

train_data=data_generator.flow_from_directory(
    dataset_path,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training',

)

val_data=data_generator.flow_from_directory(
    dataset_path,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation',

)


Found 2631 images belonging to 2 classes.
Found 291 images belonging to 2 classes.


In [3]:
model_r=tf.keras.applications.ResNet50V2(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

94683136/94668760 [==============================] - 1s 0us/step


In [4]:
for layer in model_r.layers[:-2]:
  layer.trainable=False

In [5]:
model=tf.keras.Sequential([
        model_r,
        layers.Dense(2,activation='softmax'),
])

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="PersianDetection")
config = wandb.config
config.learning_rate = 0.001

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= config.learning_rate ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [12]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=20,
          class_weight={0:1,1:3},
          callbacks=[WandbCallback()],

)

Epoch 1/20
82/82 [==============================] - 632s 7s/step - loss: 0.9193 - accuracy: 0.8510 - val_loss: 0.6450 - val_accuracy: 0.8247


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
82/82 [==============================] - 29s 356ms/step - loss: 0.5663 - accuracy: 0.9023 - val_loss: 0.6385 - val_accuracy: 0.8282
Epoch 3/20
82/82 [==============================] - 29s 352ms/step - loss: 0.3889 - accuracy: 0.9388 - val_loss: 0.5909 - val_accuracy: 0.8557
Epoch 4/20
82/82 [==============================] - 29s 354ms/step - loss: 0.2641 - accuracy: 0.9540 - val_loss: 0.4727 - val_accuracy: 0.8935
Epoch 5/20
82/82 [==============================] - 29s 356ms/step - loss: 0.2296 - accuracy: 0.9521 - val_loss: 0.4700 - val_accuracy: 0.8729
Epoch 6/20
82/82 [==============================] - 29s 354ms/step - loss: 0.1999 - accuracy: 0.9540 - val_loss: 0.3291 - val_accuracy: 0.9003
Epoch 7/20
82/82 [==============================] - 29s 354ms/step - loss: 0.1939 - accuracy: 0.9643 - val_loss: 0.8932 - val_accuracy: 0.7904
Epoch 8/20
82/82 [==============================] - 29s 352ms/step - loss: 0.2321 - accuracy: 0.9491 - val_loss: 0.3132 - val_accuracy: 0.9072

In [13]:
model.save('prsianrec.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
